In [23]:
from statistics import mode
from pymongo import MongoClient
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import json
from dotenv import load_dotenv
import os
from functools import reduce
import operator
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [24]:
load_dotenv()

True

# Lets decide the most suitable location for our company:

In this paper we will analisis 3 different preferences to decide which is the most suitable place to locate our company. 
At the end of the paper we will have an exact location for our office.

### 1rst preference: Choossing the city. 

The CEO has decided that the office should be located in one of this cities:
    
    - Madrid 
    - New York 
    - Luxemburg
    - London
    - Munich

The most important preference, that adds more value to my co-workers and the company is:

    1. A place where other startups have been founded since and during the 2009 economic crisis.
    
We are at the start of a new economic crisis due to COVID pandemic, and we think choosing a location where other companies have proven to succed during another recesion is the best place to start. 

From the dataframe df_companies we select the "city" inside the column "offices" that appears the most out of the 5 cities we have been asigned at the begging.

This first decision will determine the city we move to. 

In [25]:
conn = MongoClient("localhost:27017")
conn.list_database_names()
db = conn.get_database("ironhack")
db.list_collection_names()
companies = db.get_collection("companies")

In [26]:
df_companies = pd.DataFrame(companies.find(
        {'founded_year': {"$gt": 2008}},   
    {"name": 1, 'founded_year': 1, "offices" : 1, "_id" : 0}))

In [27]:
df_companies = df_companies.explode('offices')
df_companies.offices.isna().sum()
df_companies = df_companies.dropna()

In [28]:
df_companies["city"] = df_companies["offices"].apply(lambda o : o.get("city"))

In [29]:
selected_words = ["Madrid", "New York", "San Francisco", "London", "Hannover"] 
df_companies.city.value_counts()[selected_words]

Madrid            4
New York         39
San Francisco    50
London           27
Hannover          1
Name: city, dtype: int64

The selected city is: San Francisco

## ¿Does the coordinates of our office meet our preferences?

    - sf_latitud = 37.927728
    - sf_longitud = -122.409847

In [30]:
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [31]:
url = 'https://api.foursquare.com/v2/venues/search'

### 2nd Preference: Starbucks for our coffe breaks

Our company has a very strict policy for no eating or drinking in the office, as we work with many electrical gadgets that could be spoiled with such. For that reason, we encourage our co-workers to take coffe breaks outside the office. The company provides with Starbucks tickets, soo it is necesarry to be near many Starbucks. 

In [32]:
parametros = { "client_id" : CLIENT_ID,
              "client_secret" : CLIENT_SECRET,
              "v" : "20180323",
              "ll": "37.927728, -122.409847",
              "query": "Starbucks",
              "limit" : 50,
             }

In [33]:
response = requests.get(url , parametros)

In [34]:
starbucks = response.json().get("response").get("venues")

In [35]:
mapa_nombre = ["name"]
latitud = ["location","lat"]
longitud = ["location","lng"]

In [36]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [37]:
lista_starbucks = []
for diccionario in starbucks:
    paralista = {}
    paralista["name"] = getFromDict(diccionario,mapa_nombre)
    paralista["latitud"] = getFromDict(diccionario,latitud)
    paralista["longitud"] = getFromDict(diccionario,longitud)
    lista_starbucks.append(paralista)

In [38]:
df_starbucks = pd.DataFrame(lista_starbucks)

In [40]:
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry=gpd.points_from_xy(df_starbucks.longitud, df_starbucks.latitud))
print(f'Tipo: {type(gdf_starbucks)}')
gdf_starbucks.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Starbucks,37.919997,-122.347551,POINT (-122.34755 37.92000)
1,Starbucks,37.785980,-122.439830,POINT (-122.43983 37.78598)
2,Starbucks,37.771196,-122.389364,POINT (-122.38936 37.77120)
3,Starbucks,37.797678,-122.430455,POINT (-122.43045 37.79768)
4,Starbucks,37.786220,-122.453203,POINT (-122.45320 37.78622)


In [18]:
map_starbucks = Map(Layer(gdf_starbucks, popup_hover=[popup_element("name","Starbucks")]))

TypeError: Location should be a sized variable, for example a list or a tuple, instead got <cartoframes.viz.layer.Layer object at 0x132a7ee80> of type <class 'cartoframes.viz.layer.Layer'>.

In [19]:
office = Marker(location=[37.927728,-122.409847], tooltip = "New Office")
office.add_to(map_starbucks)
map_starbucks

NameError: name 'map_starbucks' is not defined

### 3rd Preference: The dog Pepe needs his hair done every day

The appearance of our dog is a key element of our company´s marketing strategy so it is crutial to have a haidresser near our office. 

In [162]:
parametros_2 = { "client_id" : CLIENT_ID,
              "client_secret" : CLIENT_SECRET,
              "v" : "20180323",
              "ll": "37.927728, -122.409847",
              "query": "hair",
              "limit" : 50,
             }

In [163]:
response_2 = requests.get(url , parametros_2)

In [164]:
hairdresser = response_2.json().get("response").get("venues")

In [165]:
mapa_nombre_2 = ["name"]
latitud_2 = ["location","lat"]
longitud_2 = ["location","lng"]

In [166]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [167]:
lista_hairdresser = []
for dicc in hairdresser:
    paralista_2 = {}
    paralista_2["name"] = getFromDict(dicc,mapa_nombre_2)
    paralista_2["latitud"] = getFromDict(dicc,latitud_2)
    paralista_2["longitud"] = getFromDict(dicc,longitud_2)
    lista_hairdresser.append(paralista_2)

In [22]:
df_hairdresser = pd.DataFrame(lista_hairdresser)

NameError: name 'lista_hairdresser' is not defined

In [21]:
gdf_hairdresser = gpd.GeoDataFrame(df_hairdresser, geometry=gpd.points_from_xy(df_hairdresser.longitud, df_hairdresser.latitud))
print(f'Tipo: {type(gdf_hairdresser)}')
gdf_hairdresser.head()

NameError: name 'df_hairdresser' is not defined

In [170]:
map_hairdresser = Map(Layer(gdf_hairdresser, popup_hover=[popup_element("name","Hairdresser")]))

TypeError: Location should be a sized variable, for example a list or a tuple, instead got <cartoframes.viz.layer.Layer object at 0x12d35f7c0> of type <class 'cartoframes.viz.layer.Layer'>.

In [171]:
office = Marker(location=[37.927728,-122.409847], tooltip = "New Office")
office.add_to(map_hairdresser)
map_hairdresser

NameError: name 'map_hairdresser' is not defined

# Final Decision 

We are happy with our location option: 

    - sf_latitud = 37.927728
    - sf_longitud = -122.409847
    
It meets all the required preferences we have checked. 